In [1]:
import cv2
import numpy as np

# Load the pre-trained MobileNet SSD model for vehicle detection
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt", 
    "mobilenet_iter_73000.caffemodel"
)

# Set up video capture
video_path = 'Cars Moving On Road Stock Footage - Free Download-2.mp4'  # Update the video path
cap = cv2.VideoCapture(video_path)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Parameters for Shi-Tomasi corner detection
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Conversion ratio from pixels to meters (adjust based on the actual video)
pixels_per_meter = 10

# Frame rate of the video
fps = cap.get(cv2.CAP_PROP_FPS)

# Variable to hold the previous frame's grayscale image
old_gray = None

# Variable to hold the initial points for tracking
p0 = None

# List to store the estimated speeds
speeds = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for vehicle detection
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    # Convert the frame to grayscale for optical flow
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if old_gray is None:
        # First frame: detect Shi-Tomasi corners
        old_gray = frame_gray.copy()
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

        # Skip the rest of the loop to start tracking on the next frame
        continue

    # Calculate optical flow for the detected features
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Calculate the average displacement and speed
    if len(good_new) > 0:
        displacement_pixels = np.linalg.norm(good_new - good_old, axis=1).mean()
        displacement_meters = displacement_pixels / pixels_per_meter
        speed_mps = displacement_meters * fps
        speed_kmph = speed_mps * 3.6  # Convert to km/h

        # Store the speed
        speeds.append(speed_kmph)

        # Draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = map(int, new.ravel())
            c, d = map(int, old.ravel())
            frame = cv2.line(frame, (a, b), (c, d), (0, 255, 0), 2)
            frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

        # Update previous frame and points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    # Display the frame with estimated speed
    cv2.putText(frame, f'Speed: {speed_kmph:.2f} km/h', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Speed Estimation", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


2024-10-23 16:09:09.475 Python[78325:1067278] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-23 16:09:09.475 Python[78325:1067278] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [2]:
import cv2
import numpy as np

# Load the pre-trained MobileNet SSD model for vehicle detection
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt", 
    "mobilenet_iter_73000.caffemodel"
)

# Set up video capture
video_path = 'Cars Moving On Road Stock Footage - Free Download-2.mp4'  # Update the video path to your local file
cap = cv2.VideoCapture(video_path)

# List of classes the MobileNet SSD can detect
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant",
           "sheep", "sofa", "train", "tvmonitor"]

# Set the confidence threshold for detecting vehicles
CONFIDENCE_THRESHOLD = 0.5

# Delay to slow down the video (in milliseconds)
slow_down_delay = 100  # Adjust this value to slow down the video

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for vehicle detection
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    # Loop over the detections
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections
        if confidence > CONFIDENCE_THRESHOLD:
            # Get the class label index
            idx = int(detections[0, 0, i, 1])

            # Only proceed if the detected object is a car (class label 'car')
            if CLASSES[idx] == "car":
                # Get the bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # Draw the bounding box and label on the frame
                label = f"{CLASSES[idx]}: {confidence * 100:.2f}%"
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow("Vehicle Detection", frame)

    # Slow down the video by increasing the waitKey delay
    if cv2.waitKey(slow_down_delay) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np

# Load the pre-trained MobileNet SSD model for vehicle detection
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt", 
    "mobilenet_iter_73000.caffemodel"
)

# Set up video capture
video_path = 'Cars Moving On Road Stock Footage - Free Download-2.mp4'  # Update the video path to your local file
cap = cv2.VideoCapture(video_path)

# List of classes the MobileNet SSD can detect
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant",
           "sheep", "sofa", "train", "tvmonitor"]

# Set the confidence threshold for detecting vehicles
CONFIDENCE_THRESHOLD = 0.5

# Conversion factor for pixels to meters (example value, adjust based on your video)
pixels_per_meter = 10

# Frame rate of the video
fps = cap.get(cv2.CAP_PROP_FPS)

# Dictionary to keep track of vehicle positions across frames
tracked_vehicles = {}

# Vehicle ID counter
vehicle_id_counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for vehicle detection
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    # Temporary dictionary for this frame's vehicle positions
    current_frame_vehicles = {}

    # Loop over the detections
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections
        if confidence > CONFIDENCE_THRESHOLD:
            # Get the class label index
            idx = int(detections[0, 0, i, 1])

            # Only proceed if the detected object is a car (class label 'car')
            if CLASSES[idx] == "car":
                # Get the bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # Compute the center of the bounding box
                centerX = int((startX + endX) / 2)
                centerY = int((startY + endY) / 2)

                # Try to match this detection with an existing tracked vehicle
                matched_vehicle_id = None
                for vehicle_id, prev_position in tracked_vehicles.items():
                    prev_centerX, prev_centerY = prev_position
                    distance = np.sqrt((centerX - prev_centerX) ** 2 + (centerY - prev_centerY) ** 2)
                    if distance < 50:  # Threshold to match the same vehicle
                        matched_vehicle_id = vehicle_id
                        break

                # If no match, assign a new vehicle ID
                if matched_vehicle_id is None:
                    matched_vehicle_id = vehicle_id_counter
                    vehicle_id_counter += 1

                # Update current frame vehicle positions
                current_frame_vehicles[matched_vehicle_id] = (centerX, centerY)

                # Measure the speed if the vehicle was previously tracked
                if matched_vehicle_id in tracked_vehicles:
                    prev_centerX, prev_centerY = tracked_vehicles[matched_vehicle_id]
                    displacement_pixels = np.sqrt((centerX - prev_centerX) ** 2 + (centerY - prev_centerY) ** 2)
                    displacement_meters = displacement_pixels / pixels_per_meter
                    speed_mps = displacement_meters * fps
                    speed_kmph = speed_mps * 3.6  # Convert to km/h

                    # Draw the speed and bounding box on the frame
                    label = f"ID {matched_vehicle_id} Speed: {speed_kmph:.2f} km/h"
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Update the tracked vehicles with current frame positions
    tracked_vehicles = current_frame_vehicles

    # Display the output frame
    cv2.imshow("Vehicle Detection and Speed Estimation", frame)

    # Exit on 'q' key press
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import time
from collections import deque

# Load the Caffe model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "mobilenet_iter_73000.caffemodel")

# Initialize video capture
video_path = 'Cars Moving On Road Stock Footage - Free Download-2.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
scale_factor = 10  # Example scale factor for pixel-to-meter conversion (adjust based on calibration)

# Function to estimate speed
def calculate_speed(pixel_distance, scale_factor, frame_rate):
    # Convert pixel displacement to meters
    real_world_distance = pixel_distance / scale_factor
    # Speed in meters per second
    speed_mps = real_world_distance * frame_rate
    # Convert to kilometers per hour
    speed_kph = speed_mps * 3.6
    return speed_kph

# Vehicle tracking
tracker = {}
track_history = deque(maxlen=5)  # Store last 5 positions for each tracked object

# Process each frame from the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for object detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, scalefactor=0.007843, size=(300, 300), mean=(127.5, 127.5, 127.5))
    net.setInput(blob)

    # Perform object detection
    detections = net.forward()

    # Track detected vehicles using centroids
    current_centroids = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])
            # Filter for vehicles: assuming class IDs correspond to car-related classes in your model
            if class_id == 7:  # Example: COCO class ID for car
                # Get bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                current_centroids.append(centroid)

                # Draw the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Confidence: {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Update tracker and calculate speed
    for i, centroid in enumerate(current_centroids):
        if i not in tracker:
            tracker[i] = centroid
            track_history.append((i, centroid))
        else:
            previous_centroid = tracker[i]
            pixel_distance = np.linalg.norm(np.array(centroid) - np.array(previous_centroid))

            # Estimate speed
            speed_kph = calculate_speed(pixel_distance, scale_factor, fps)
            cv2.putText(frame, f'Speed: {speed_kph:.2f} km/h', (centroid[0], centroid[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

            # Update tracker
            tracker[i] = centroid
            track_history.append((i, centroid))

    # Display the frame
    cv2.imshow('Vehicle Detection and Speed Estimation', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
from collections import deque

# Load the Caffe model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "mobilenet_iter_73000.caffemodel")

# Initialize video capture
video_path = 'Cars Moving On Road Stock Footage - Free Download-2.mp4'  # Change to the path of your video file
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Set a scale factor for pixel-to-meter conversion
# Example: 50 pixels correspond to 5 meters => scale_factor = 50 / 5 = 10 pixels per meter
scale_factor = 10.0  # Adjust this based on your calibration

# Function to estimate speed
def calculate_speed(pixel_distance, scale_factor, frame_rate):
    # Convert pixel displacement to meters
    real_world_distance = pixel_distance / scale_factor
    # Speed in meters per second
    speed_mps = real_world_distance * frame_rate
    # Convert to kilometers per hour
    speed_kph = speed_mps * 3.6
    return speed_kph

# Function to smooth the speed estimates
def smoothed_speed(new_speed, speed_history):
    speed_history.append(new_speed)
    return sum(speed_history) / len(speed_history)

# Vehicle tracking
tracker = {}
speed_history = deque(maxlen=5)  # Keep the last 5 speed measurements

# Process each frame from the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for object detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, scalefactor=0.007843, size=(300, 300), mean=(127.5, 127.5, 127.5))
    net.setInput(blob)

    # Perform object detection
    detections = net.forward()

    # Track detected vehicles using centroids
    current_centroids = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])
            # Assuming class ID 7 corresponds to a car (check your model's labels)
            if class_id == 7:
                # Get bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                current_centroids.append(centroid)

                # Draw the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Confidence: {confidence:.2f}', (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Update tracker and calculate speed
    for i, centroid in enumerate(current_centroids):
        if i not in tracker:
            tracker[i] = centroid
        else:
            previous_centroid = tracker[i]
            pixel_distance = np.linalg.norm(np.array(centroid) - np.array(previous_centroid))

            # Estimate speed
            speed_kph = calculate_speed(pixel_distance, scale_factor, fps)

            # Apply smoothing to reduce variability
            smoothed_speed_kph = smoothed_speed(speed_kph, speed_history)

            # Display the smoothed speed
            cv2.putText(frame, f'Speed: {smoothed_speed_kph:.2f} km/h', (centroid[0], centroid[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

            # Update tracker
            tracker[i] = centroid

    # Display the frame
    cv2.imshow('Vehicle Detection and Speed Estimation', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


2024-10-21 21:52:56.039 Python[75779:808207] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-21 21:52:56.039 Python[75779:808207] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [5]:
import cv2
import numpy as np
from collections import deque

# Load the Caffe model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "mobilenet_iter_73000.caffemodel")

# Use the uploaded video file
video_path = 'Cars Moving On Road Stock Footage - Free Download-2.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Set a scale factor for pixel-to-meter conversion
# The scale_factor needs calibration based on a known distance in the video
scale_factor = 33.33  # Adjust this based on your calibration

# Function to estimate speed
def calculate_speed(pixel_distance, scale_factor, frame_rate):
    real_world_distance = pixel_distance / scale_factor
    speed_mps = real_world_distance * frame_rate
    speed_kph = speed_mps * 3.6
    return speed_kph

# Function to smooth the speed estimates
def smoothed_speed(new_speed, speed_history):
    speed_history.append(new_speed)
    return sum(speed_history) / len(speed_history)

# Vehicle tracking
tracker = {}
speed_history = deque(maxlen=5)  # Keep the last 5 speed measurements

# Process each frame from the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for object detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, scalefactor=0.007843, size=(300, 300), mean=(127.5, 127.5, 127.5))
    net.setInput(blob)

    # Perform object detection
    detections = net.forward()

    # Track detected vehicles using centroids
    current_centroids = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])
            # Assuming class ID 7 corresponds to a car
            if class_id == 7:
                # Get bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                current_centroids.append(centroid)

                # Draw the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Confidence: {confidence:.2f}', (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Update tracker and calculate speed
    for i, centroid in enumerate(current_centroids):
        if i not in tracker:
            tracker[i] = centroid
        else:
            previous_centroid = tracker[i]
            pixel_distance = np.linalg.norm(np.array(centroid) - np.array(previous_centroid))

            # Estimate speed
            speed_kph = calculate_speed(pixel_distance, scale_factor, fps)

            # Apply smoothing to reduce variability
            smoothed_speed_kph = smoothed_speed(speed_kph, speed_history)

            # Display the smoothed speed
            cv2.putText(frame, f'Speed: {smoothed_speed_kph:.2f} km/h', (centroid[0], centroid[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

            # Update tracker
            tracker[i] = centroid

    # Display the frame
    cv2.imshow('Vehicle Detection and Speed Estimation', frame)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
from collections import deque

# Load the Caffe model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "mobilenet_iter_73000.caffemodel")

# Use the uploaded video file
video_path = 'Cars Moving On Road Stock Footage - Free Download-2.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Estimated scale factor based on an average car length of 4.5 meters and approximately 150 pixels
scale_factor = 33.33  # Pixels per meter

# Define the y-coordinate of the target line where speed will be shown
target_line_y = 400  # Adjust this value based on the position in the frame

# Function to estimate speed
def calculate_speed(pixel_distance, scale_factor, frame_rate):
    real_world_distance = pixel_distance / scale_factor
    speed_mps = real_world_distance * frame_rate
    speed_kph = speed_mps * 3.6
    return speed_kph

# Function to smooth the speed estimates
def smoothed_speed(new_speed, speed_history):
    speed_history.append(new_speed)
    return sum(speed_history) / len(speed_history)

# Vehicle tracking
tracker = {}
speed_history = deque(maxlen=5)  # Keep the last 5 speed measurements

# Process each frame from the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Draw the target line
    cv2.line(frame, (0, target_line_y), (frame.shape[1], target_line_y), (0, 0, 255), 2)

    # Prepare the frame for object detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, scalefactor=0.007843, size=(300, 300), mean=(127.5, 127.5, 127.5))
    net.setInput(blob)

    # Perform object detection
    detections = net.forward()

    # Track detected vehicles using centroids
    current_centroids = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])
            # Assuming class ID 7 corresponds to a car
            if class_id == 7:
                # Get bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
                current_centroids.append((i, centroid))  # Store the index and centroid

                # Draw the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Confidence: {confidence:.2f}', (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Update tracker and calculate speed
    for i, centroid in current_centroids:
        if i not in tracker:
            tracker[i] = centroid
        else:
            previous_centroid = tracker[i]
            pixel_distance = np.linalg.norm(np.array(centroid) - np.array(previous_centroid))

            # Estimate speed
            speed_kph = calculate_speed(pixel_distance, scale_factor, fps)

            # Apply smoothing to reduce variability
            smoothed_speed_kph = smoothed_speed(speed_kph, speed_history)

            # Display the speed only if the centroid crosses the target line
            if previous_centroid[1] < target_line_y <= centroid[1]:
                cv2.putText(frame, f'Speed: {smoothed_speed_kph:.2f} km/h', (centroid[0], centroid[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

            # Update tracker
            tracker[i] = centroid

    # Display the frame
    cv2.imshow('Vehicle Detection and Speed Estimation', frame)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


2024-10-23 15:54:37.292 Python[78280:1064755] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-23 15:54:37.292 Python[78280:1064755] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
